# Exam 1, Julia Course 2023

## Load Packages and printmat

In [5]:
using Printf, Dates, Statistics, JLD2, Plots
include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

## Load Data and Calculate Returns

from the jld2 file Fx5min.jld2. You want the `dN` (vector of julia DateTimes, eg `2020-01-01T00:05:00`) and `USD_EUR` (vector with the exchange rate: the number of USD you pay for 1 EUR). The data are 5 minute data so there are 288 elements per day for each day 2000-2022. There are 315,648 observations.

Calculate a vector of log returns, that is `log(price/price 5 minute earlier)`.

In [8]:
# Load the file into a variable
file_data = load("./Fx5min/Fx5min.jld2")

# List all variables in the file
keys(file_data)

KeySet for a Dict{String, Any} with 2 entries. Keys:
  "USD_EUR"
  "dN"

# Estimate the Std for each Hour

Extract all data points that refer to a time between 00:00 (midnight) and 00:59 (just before 1 o'clock in the night). Estimate the standard deviation of the returns. Repeat for 01:00 and 01:59, etc. Plot the results (multiplied by `10_000`.) with the hour on the horizontal axis. (Hint: `hour(a DateTime) == 0`.

# Estimate Autoregressions for each Day of Data

Consider the regression

$r_t = a + b r_{t-1} + \epsilon_t$,

where $r_t$ is a 5-minute return and $r_{t-1}$ is the previous 5-minute return.

Extract data for day 1, estimate the regression and save the estimate of $b$. Redo for day 2, etc.  (Hint:`unique(Date.(vector of DateTimes)).` It's OK if the regressor $r_{t-1}$ includes the observation from 23:55 the day before.  

Create a moving average over `b` for the current and previous 5 days. Plot the `b` estimates against dates (on the horizonztal axis) as well as the moving average (in the same plot) 

# Plot a Histogram of the `b` Estimates